# Simulation of EGF stimulation in drug-adapted BRAF<sup>V600E</sup> melanoma cells using the MARM1 model

Here you can simulate the time-course respose of drug-adapted BRAF<sup>V600E</sup> melanoma cells to addition of exogenous EGF. This code simulates the time-course response of A375 melanoma cells adapted to a dose of RAF (vemurafenib) and/or MEK inhibitors (cobimetinib) and then stimulated with EGF.

**Note**: this code performs the simulation for a single condition and visualizes the time-course response. Use the Jupyter Notebook *MARM1_simulation_multiple_conditions.ipynb* to generate simulation results for multiple conditions (e.g. multiple dose combinations). 



## Import of libraries
Importing libraries necessary to run MARM1 model simulations.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing.pool import Pool
import multiprocessing
import copy as cp
from tqdm.notebook import tqdm, trange
import time
import math
import random
import itertools

Importing the MARM1 PySB model and the simulator.  

In [2]:
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from pysb.bng import generate_equations

from MARM2_nE_SBa_Sparse import model

## User-defined experimental setup
In this section you can alter the setup of the experiment simulated by MARM1. First, you need to define the experimental setup of the pre-treatment phase and of the subsequent ligand stimulation phase. The variables needed for the pre-treatment phase are:

<b>RAFi_concentration ($\mu$M)</b>: defines the concentration of the RAF inhibitor (vemurafenib) during the pre-treatment and treatment phase. 

<b>MEKi_concentration ($\mu$M)</b>: defines the concentration of the RAF inhibitor (vemurafenib) during the pre-treatment and treatment phase. 

<b>Pretreatment_time (h)</b>: defines the duration of the pre-treatment phase.
    
For the treatment phase, you need to set the concentration of the EGF ligand and the running time of simulation after ligand stimulation. This is done with the following variables:

<b>EGF_concentration (ng/mL)</b>: defines the EGF concentrations used to stimulate cells. We have calibrated the model only using data from 100 ng/mL, which is thus the default value.

<b>Simulation_time (h)</b>: define the simulation time after ligand stimulation. For the stimulation of cells expressing normal levels of EGFR, 2 hours is enough to visualize the pulsatile reactivation of MAPK signaling.

<b>N_time_points</b>: define the number of time points returned by each model simulation. 

Choose the experiment to simulate by setting the following variables and re-running the notebook:

1. **PRAF inhibitor** pre-treatment concentration in μM.

2. **MEK inhibitor** pre-treatment concentration in μM.

3. **Pretreatment duration** in hours.

In [3]:
t_pretrt = 24

5. **Simulation time** in hours.

In [4]:
t_trt = 24

6. **Parameter set** selects which of the 50 best-fit parameter sets to use for the simulation. Set 0 is the best fit and 49 the worst.

In [5]:
param_set_index = 13
# param_set_index = 0

7. **N_time_points** defineds the number of time points returned by each individual model simulation

In [6]:
N_time_points = 97

## Generate model equations
PySB runs BioNetGen to generate the reaction network

In [7]:
generate_equations(model)
#[i.name for i in model.parameters]

## Parameter set preparation

Override the `EGFR_crispr` parameter with the user-specified value for EGFR under/over-expression and set naive-treatment conditions for concentrations of RAF and MEK inhibitors and EGF.

In [8]:
param_sets = pd.read_csv('RTKERK_pRAF_EGF_EGFR_MEKi_PRAFi_RAFi.csv', index_col=0)
# finds the parameters of the .csv file that correspond to Cobimetinib and Vemurafenib and maps them to MEKi and RAFi (respectively)
rename_dict = {}
for i in param_sets.columns:
    if "Cobimetinib" in i or "Vemurafenib" in i:
        rename_dict[i] = i.replace("Cobimetinib","MEKi").replace("Vemurafenib","RAFi")
param_sets = param_sets.rename(columns = rename_dict)

# finds the parameters which are stored in .csv file but not in the model and removes them
csv_spec_params = set(param_sets.columns)-(set(param_sets.columns)&set([i.name for i in model.parameters]))
param_sets = param_sets.drop(csv_spec_params, axis=1)

# Reduces GTP hydrolysis to align with expected GTPase activity of NRASQ61mut
#param_sets["catalyze_NF1_RAS_gdp_kcatr"] = param_sets["catalyze_NF1_RAS_gdp_kcatr"] / 10
# Makes dimers less favorable
#param_sets["ep_RAF_RAF_mod_RASgtp_double_ddG"] = param_sets["ep_RAF_RAF_mod_RASgtp_double_ddG"]/5
# Reduces removes preference against second inhibitor binding to model pan RAF inhibitor
#param_sets["ep_RAF_RAF_mod_RAFi_double_ddG"] = 0
# Remove negative feedback CRAF
#param_sets['ep_RAF_RAF_mod_pRAF_ddG'] = 0 
# Sets 0 SOS1_0 to prevent pathway activity through non BRAFV600E
#param_sets['SOS1_0'] = 0


params = param_sets.iloc[param_set_index].to_dict()

In [9]:
#params['EGF_0'] = 0.0
params['RAFi_0'] = 0.0
params['MEKi_0'] = 0.0

In [10]:
#print(model.parameters.NRAS_Q61mut.value)
#print(model.parameters.catalyze_NF1_RAS_gdp_kcatr.value)
#print(model.parameters.q61_RAS_gtp_kcat.value)
#print(model.parameters.ep_RAF_RAF_mod_RAFi_single_ddG.value)
#print(model.parameters.ep_RAF_RAF_mod_RAFi_double_ddG.value)
#print(model.parameters.bind_PRAFi_RAF_kf.value)

## Simulations

First we define some utility functions that will be used below.
*Equilibrate* runs a model simulation till steady state for that parameter set.
*Get_species_index* find and retunrs the index of speies in the model given input specie patterns. 

In [11]:
def equilibrate(simulator, initials,verbose = True):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 10
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        if verbose:
            print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(simulator.model.species)
        if verbose:
            print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [12]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

## Initial equilibrium
First we run the model from its baseline initial conditions until equilibrium is reached. For example protein synthesis/degradation, phosphorylation/dephosphorylation, and drug binding/unbinding all need to reach steady state to match the state of the cells in the experimental setup. There may be some time without visible progress as behind the scenes PySB runs BioNetGen to generate the reaction network and Cython to compile the resulting differential equations into efficient executable code.

In [13]:
sim = ScipyOdeSimulator(model,param_values=params) 
df_eq = equilibrate(sim, None)

    at t=100   ... 875/875 species converged


Now that the model has been simulated once and the actual molecular species have been enumerated, we can find the exact species numbers for the inhibitors and EGF. These are needed so that their concentrations can be overridden in the model state for subsequent simulations.

In [14]:
RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None)**model.compartments.CP)
MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None)**model.compartments.CP)
#EGF_index = get_species_index(model, model.monomers.EGF(rtk=None)**model.compartments.CP)

## Inhibitor pre-treatment

We take the final state of the equilibration simulation and use it as the initial state of this new simulation, overriding the RAFi and MEKi concentrations with the user-selected values.

In [15]:
initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
initials_pre[RAFi_index] = 0.0
initials_pre[MEKi_index] = 0.0
#initials_pre[EGF_index] = 0.0

#fixed time pre-treatment simulation
tspan_pretrt = np.linspace(0, t_pretrt, N_time_points)
df_pre=sim.run(tspan=tspan_pretrt, initials=initials_pre.to_list()).dataframe

#run pre-tretment to steady state instead of using specified time  
#df_pre = equilibrate(sim, initials_pre)

In case the previous simulation was run to steady state, we want to retain only the first t_pretrt hours of pre-treatment plus the state at final equilibrium. So we cut the time series down using a Pandas slice operation and adjust the remaining time values to begin at -pre_time_max.

In [16]:
if (len(df_pre.loc[:t_pretrt])<len(df_pre)):
   df_pre_tmp = df_pre.loc[:t_pretrt]
   df_pre_tmp.iloc[-1] = df_pre.iloc[-1]
   df_pre= df_pre_tmp
df_pre['time'] = df_pre.index
df_pre['time'] = df_pre['time']-t_pretrt
df_pre['time'].iloc[-1] = 0
df_pre.reset_index(drop=True, inplace=True)
df_pre.set_index('time', inplace=True)

## Inhibitor treatment

We run another simulation starting from the final state of the pre-treatment simulation, overriding the MEKi and PRAFi concentrations with the user-selected values. This is a fixed-time simulation rather than the steady-state equilibration used in the previous simulations.

In [17]:
#set the dilution range for the PRAF inhibitor, which is x axis
#RAFi_dil=np.logspace(-4, 1, 9); #uM
RAFi_dil=np.logspace(-2.25,.5, 9); #uM
RAFi_dil = np.concatenate(([0],RAFi_dil))
#set the dilution range for the MEK inhibitor, each value generates a curve
#MEKi_dil=np.logspace(-4, 1, 7); #uM
MEKi_dil=np.logspace(-2.75,0, 9); #uM
MEKi_dil=np.concatenate(([0],MEKi_dil))
print(MEKi_dil)
#set the values of f and g to model RAF inhibitors with different complex drug-protein interactions
#1st generation: f= 0.001, g=1000; panRAF: f= 0.001, g=1; dimer selective f=1, g=0.001
# Moved from using f and g to single and double ddG values (respectively), currently reads from model, but can be perturbed if needed
s_ddG=[params["ep_RAF_RAF_mod_RAFi_single_ddG"][0]];
print(s_ddG)
d_ddG=[params["ep_RAF_RAF_mod_RAFi_double_ddG"][0]];
print(d_ddG)
fgtitle=['pan_RAF'];

[0.         0.00177828 0.00392419 0.00865964 0.01910953 0.04216965
 0.0930572  0.2053525  0.45315836 1.        ]
[-5.697253636]
[11.05981345]


In [18]:
multiprocessing.cpu_count()

28

In [19]:
def simulate_inhib_dose(dose_info):
    params.update({'RAFi_0': dose_info[0],'MEKi_0': dose_info[1]});
    #run this to assure model is run to steady_ state, 
    res = equilibrate(ScipyOdeSimulator(model,param_values=params) , None,verbose=False)
    print(1,end="")
    return [dose_info, res.iloc[-1]]

In [21]:
if __name__ == '__main__':
    #create a bar to keep track of simulation progress
    #define observables to plot, handle plotting alias with dict
    plt_obs=['pMEK', 'pMEK_obs', 'pERK', 'pERK_obs'];
    #plot_obs_names=['pMEK', 'pMEK/tMEK', 'pERK', 'pERK/tERK'];
    #plot_name_dict = dict(zip(plt_obs,plot_obs_names))
    dr_df = pd.DataFrame(columns = ["RAFi_0_uM","MEKi_0_uM"]+plt_obs)
    dose_ind = list(itertools.product(*[RAFi_dil,MEKi_dil]))
    #print(dose_ind)
    dr_df["RAFi_0_uM"] = [i[0] for i in dose_ind]
    dr_df["MEKi_0_uM"] = [i[1] for i in dose_ind]
    dr_df = dr_df.set_index(["RAFi_0_uM","MEKi_0_uM"])
    # only 14 cpu's are availible 
    pol = Pool(processes=12)
    results = pol.map(simulate_inhib_dose, [i for i in dose_ind])
    for i in range(len(results)):
        dr_df.loc[results[i][0]] = results[i][1][plt_obs]
    dr_df.to_csv("dose_response_df/brange_RAFi_MEKi_BRAFV600E")
    pol.close()
    pol.join()
    

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [ ]:
!conda env export --name test_env